In [2]:
#=====================================================================
# elasticsearch에 한국어 tokenizer 테스트 예시 
# 참고 : https://jvvp.tistory.com/1158
#       https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=ysulshin&logNo=220932285051
#=====================================================================
import pprint

from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch("http://192.168.0.130:9200/")
es.info()

C:\Users\bong9\anaconda3\envs\pytorch\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': 'lenovo-x240',
 'cluster_name': 'mpower',
 'cluster_uuid': 'JJ1h3dNTRvSLJFW3gj5MCw',
 'version': {'number': '7.17.3',
  'build_flavor': 'default',
  'build_type': 'zip',
  'build_hash': '5ad023604c8d7416c9eb6c0eadb62b14e766caff',
  'build_date': '2022-04-19T08:11:19.070913226Z',
  'build_snapshot': False,
  'lucene_version': '8.11.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [3]:
# nori_tokenizer(한국어 Mecab 토큰너나이저) 동작하는지테스트 해봄
body = {
    "tokenizer":"nori_tokenizer",
    "filter" : [
        "lowercase",
        "stop",
        "snowball"
    ],
    "text" : ["매일 비가 오네요"]
}
es.indices.analyze(body=body)


{'tokens': [{'token': '매일',
   'start_offset': 0,
   'end_offset': 2,
   'type': 'word',
   'position': 0},
  {'token': '비',
   'start_offset': 3,
   'end_offset': 4,
   'type': 'word',
   'position': 1},
  {'token': '가',
   'start_offset': 4,
   'end_offset': 5,
   'type': 'word',
   'position': 2},
  {'token': '오',
   'start_offset': 6,
   'end_offset': 7,
   'type': 'word',
   'position': 3},
  {'token': '네요',
   'start_offset': 7,
   'end_offset': 9,
   'type': 'word',
   'position': 4}]}

In [4]:
###########################################################
# 인덱스 생성/삭제
###########################################################
## 인덱스 생성
def create_index(index, mapping=None):
    if not es.indices.exists(index=index):
        return es.indices.create(index=index ,body=mapping)
    
## 인덱스 자체 삭제
def delete_index(index):
    if es.indices.exists(index=index):
        return es.indices.delete(index=index)
    
###########################################################
# 인데스에 데이터 추가 
###########################################################
def insert(index, doc_type, body):
    return es.index(index=index, doc_type=doc_type, body=body)



In [5]:
import json

INDEX_NAME = "mytoken"

# .json 파일을 불러옴
JSON_FILE = 'tokenizer_setting.json'

with open(JSON_FILE, "r", encoding="utf-8") as f:
    setting = json.load(f)
    
print(setting)
print('\n')

# 기존에 index 있으면 삭제 
res=delete_index(index=INDEX_NAME)
print(res)
print('\n')

# index 생성

res=create_index(index=INDEX_NAME, mapping=setting)
print(res)
print('\n')

result=es.indices.get_settings(index=INDEX_NAME)
pprint.pprint(result)
print('\n')



{'settings': {'number_of_shards': 2, 'number_of_replicas': 1, 'analysis': {'analyzer': {'mytokenizer': {'type': 'custom', 'tokenizer': 'nori_tokenizer', 'decompound_mode': 'mixed'}}}}, 'mappings': {'dynamic': 'true', '_source': {'enabled': 'true'}, 'properties': {'title': {'type': 'text', 'analyzer': 'mytokenizer'}, 'content': {'type': 'text', 'analyzer': 'mytokenizer'}}}}


{'acknowledged': True}


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'mytoken'}


{'mytoken': {'settings': {'index': {'analysis': {'analyzer': {'mytokenizer': {'decompound_mode': 'mixed',
                                                                              'tokenizer': 'nori_tokenizer',
                                                                              'type': 'custom'}}},
                                    'creation_date': '1653357095254',
                                    'number_of_replicas': '1',
                                    'number_of_shards': '2',
               

In [90]:
# 생성한 mytoken 인덱스에 mytokenizer 토큰너나이저 이용해서 테스트 
body = {
    "analyzer": "mytokenizer",
    "text": "제주도 개발 공사"
}

INDEX_NAME = "mytoken"
es.indices.analyze(index=INDEX_NAME,body=body)


{'tokens': [{'token': '제주',
   'start_offset': 0,
   'end_offset': 2,
   'type': 'word',
   'position': 0},
  {'token': '도',
   'start_offset': 2,
   'end_offset': 3,
   'type': 'word',
   'position': 1},
  {'token': '개발',
   'start_offset': 4,
   'end_offset': 6,
   'type': 'word',
   'position': 2},
  {'token': '공사',
   'start_offset': 7,
   'end_offset': 9,
   'type': 'word',
   'position': 3}]}

In [87]:
body = {
    "text": "제주도 개발 공사"
}

INDEX_NAME = "mytoken"
es.indices.analyze(index=INDEX_NAME,body=body)

{'tokens': [{'token': '제주도',
   'start_offset': 0,
   'end_offset': 3,
   'type': '<HANGUL>',
   'position': 0},
  {'token': '개발',
   'start_offset': 4,
   'end_offset': 6,
   'type': '<HANGUL>',
   'position': 1},
  {'token': '공사',
   'start_offset': 7,
   'end_offset': 9,
   'type': '<HANGUL>',
   'position': 2}]}

In [8]:
contents = [
    {
        "id": 1,
        "title": "제주 관광품질 개선·브랜드 강화 급선무",
        "content": "제주 관광산업의 지속성장 가능성을 확보하기 위해 관광산업 다각화, 고부가가치화 등이 요구되고 있는 가운데 바가지요금·불친절 등 제주 관광품질 개선이 선행 과제로 제시됐다. 특히 세계적인 방역조치 완화로 국가간 관광객 유치경쟁이 심화될 것으로 예상되는 만큼, 관광품질 향상과 제주 브랜드 강화를 통해 제주관광 재도약의 기회로 삼아야 한다는 주문이다."
    },
    {
        "id": 2,
        "title": "제주도, 요금 부풀린 어르신 행복택시 엄중 조치",
        "content": "어르신 행복택시 요금 부풀리기로 보조금을 챙긴 제주도내 택시회사 등을 상대로 환수조치 등 강력한 제재를 가할 전망이다."
    },
    {
        "id": 3,
        "title": "들뜬 발걸음…제주 학교 일상회복에 '활기'",
        "content": "코로나19 상황 속 이뤄지던 기존 학사운영이 폐지되고 23일부터 도내 전체 학교의 전면 등교 수업이 시행되면서 교육 현장이 활기를 띠고 있다."
    },
    {
        "id": 4,
        "title": "도개발공사…마음에온 행복주택 예비입주자 모집",
        "content": "제주개발공사는 마음에온 행복주택의 예비입주자를 모집한다고 23일 밝혔다.이번 모집은 공사에서 운영 중인 행복주택의 예비입주자를 모집하기 위한 것으로 5개 단지에 167세대이다."
    },
]

In [9]:
# 인덱스에 contents 추가 
DOC_TYPE = "_doc"  # ElasticSearch 7.0 부터는 doc_type 개념이 사라지고, _doc을 사용해야 함
INDEX_NAME = "mytoken"


for content in contents:
    res = insert(index=INDEX_NAME, doc_type=DOC_TYPE, body=content) 
    print(res)
    print("\n")

{'_index': 'mytoken', '_type': '_doc', '_id': 'XlLD84ABWwNQ7PQ9WAy2', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}


{'_index': 'mytoken', '_type': '_doc', '_id': 'X1LD84ABWwNQ7PQ9WQxp', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}


{'_index': 'mytoken', '_type': '_doc', '_id': 'YFLD84ABWwNQ7PQ9WQx8', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}


{'_index': 'mytoken', '_type': '_doc', '_id': 'YVLD84ABWwNQ7PQ9WQyQ', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 2, '_primary_term': 1}




In [61]:
#####################################################################
# 다른 인덱스에 값을 여러개 검색하는 예시
# => msearch 함수 이용
# => 필드 title, content 등은 수정 필요 
#####################################################################
def bi_index_search(index1, index2, keyword):
    
    query = [
        {"index": index1},     # 1번째 인덱스
        {"query": {"match": {'title': keyword}}}, # 검색 조건
        {"index": index2},     # 2번째 인덱스
        {"query": {"match": {'content': keyword}}} # 검색 조건
    ]
    
    return es.msearch(body=query)
#####################################################################

keyword='재도약'

INDEX_NAME="mytoken"

res=bi_index_search(index1=INDEX_NAME, index2=INDEX_NAME, keyword=keyword)
print('res_len:{}\n'.format(len(res['responses'])))

for response in res['responses']:
    print(response['hits']['hits'])
    print('\n')

res_len:2

[]


[{'_index': 'mytoken', '_type': '_doc', '_id': 'XlLD84ABWwNQ7PQ9WAy2', '_score': 1.6022167, '_source': {'id': 1, 'title': '제주 관광품질 개선·브랜드 강화 급선무', 'content': '제주 관광산업의 지속성장 가능성을 확보하기 위해 관광산업 다각화, 고부가가치화 등이 요구되고 있는 가운데 바가지요금·불친절 등 제주 관광품질 개선이 선행 과제로 제시됐다. 특히 세계적인 방역조치 완화로 국가간 관광객 유치경쟁이 심화될 것으로 예상되는 만큼, 관광품질 향상과 제주 브랜드 강화를 통해 제주관광 재도약의 기회로 삼아야 한다는 주문이다.'}}]




In [62]:
###############################################################
# 멀티 필드 검색 하는 경우 예시 
#  => multi_match 이용
#  => 필드 title, content 등은 수정 필요 
###############################################################
def multi_field_search(index, keyword):
    
    query = {
        "query": { 
            "multi_match": {
                "query": keyword,
                "fields": ["title", "content"]
            }
        }
    }
    
    return es.search(index=index,body=query)
###############################################################   


keyword='제주'
INDEX_NAME="mytoken"


res=multi_field_search(index=INDEX_NAME, keyword=keyword)
print('res_len:{}\n'.format(len(res['hits']['hits'])))

for response in res['hits']['hits']:
    print(response)
    print('\n')

res_len:4

{'_index': 'mytoken', '_type': '_doc', '_id': 'XlLD84ABWwNQ7PQ9WAy2', '_score': 0.51783925, '_source': {'id': 1, 'title': '제주 관광품질 개선·브랜드 강화 급선무', 'content': '제주 관광산업의 지속성장 가능성을 확보하기 위해 관광산업 다각화, 고부가가치화 등이 요구되고 있는 가운데 바가지요금·불친절 등 제주 관광품질 개선이 선행 과제로 제시됐다. 특히 세계적인 방역조치 완화로 국가간 관광객 유치경쟁이 심화될 것으로 예상되는 만큼, 관광품질 향상과 제주 브랜드 강화를 통해 제주관광 재도약의 기회로 삼아야 한다는 주문이다.'}}


{'_index': 'mytoken', '_type': '_doc', '_id': 'X1LD84ABWwNQ7PQ9WQxp', '_score': 0.47628897, '_source': {'id': 2, 'title': '제주도, 요금 부풀린 어르신 행복택시 엄중 조치', 'content': '어르신 행복택시 요금 부풀리기로 보조금을 챙긴 제주도내 택시회사 등을 상대로 환수조치 등 강력한 제재를 가할 전망이다.'}}


{'_index': 'mytoken', '_type': '_doc', '_id': 'YFLD84ABWwNQ7PQ9WQx8', '_score': 0.2876821, '_source': {'id': 3, 'title': "들뜬 발걸음…제주 학교 일상회복에 '활기'", 'content': '코로나19 상황 속 이뤄지던 기존 학사운영이 폐지되고 23일부터 도내 전체 학교의 전면 등교 수업이 시행되면서 교육 현장이 활기를 띠고 있다.'}}


{'_index': 'mytoken', '_type': '_doc', '_id': 'YVLD84ABWwNQ7PQ9WQyQ', '_score': 0.14097276, '_source': {'id': 4, 'title': '도개발공사…마음에온 행복주택 예비입주자 모집',

In [81]:
# term 쿼리 
# => keyword를 분석하지 않고(toeknizer 하지 않고) 그냥 입력된 keyword 그대로 쿼리 하는 것.(*스코어는 1.0)
INDEX_NAME = "mytoken"
keyword = "제주도"

query = {
    "query" : {
        "term" : {"title": keyword}
    }
}

# match 는 분석해서 keyword 함
query1 = {
    "query" : {
        "match" : {"title": keyword}
    }
}

res = es.search(index=INDEX_NAME, body=query)
print('res_len:{}\n'.format(len(res['hits']['hits'])))

for response in res['hits']['hits']:
    print(response)
    print('\n')


res_len:0



In [78]:
# terms 쿼리 
# => keyword를 분석하지 않고(toeknizer 하지 않고) 그냥 입력된 keyword 그대로 쿼리 하는 것.(*스코어는 1.0)
# => 검색어가 2개이상인 경우(여러단어를 찾는경우 = 기본은 or 연산)

INDEX_NAME = "mytoken"
keyword1 = "제주도"
keyword2 = "주택"
keyword3 = "예비입주"

query = {
    "query" : {
        "terms" : {"title": [keyword1, keyword2, keyword3]}
    }
}

res = es.search(index=INDEX_NAME, body=query)
print('res_len:{}\n'.format(len(res['hits']['hits'])))

for response in res['hits']['hits']:
    print(response)
    print('\n')


res_len:1

{'_index': 'mytoken', '_type': '_doc', '_id': 'YVLD84ABWwNQ7PQ9WQyQ', '_score': 1.0, '_source': {'id': 4, 'title': '도개발공사…마음에온 행복주택 예비입주자 모집', 'content': '제주개발공사는 마음에온 행복주택의 예비입주자를 모집한다고 23일 밝혔다.이번 모집은 공사에서 운영 중인 행복주택의 예비입주자를 모집하기 위한 것으로 5개 단지에 167세대이다.'}}


